<h3>Importing The Necessary Libraries

In [6]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import csv

Getting the chrome driver for automation

In [7]:
driver=webdriver.Chrome()

Getting the Website

In [8]:
driver.get("https://collegedunia.com/india-colleges?custom_params=%5Bview%3Atable%5D")

In [9]:
# printing the title
print(driver.title)

Top Colleges in India - 2023 Rankings, Fees, Placements - Collegedunia


<h3> Scraping Code

In [ ]:
start =time.time()
count=0
college=[]
target_scroll = 'jsx-2796823646.jsx-1342907234.endOfContainer'
with open('college.csv',mode='a',newline='',encoding='utf-8') as csv_file:
    csv_writer=csv.writer(csv_file)
    csv_writer.writerow(["Cd_Rank","college_name","fees", "placement", "review", "rank"])
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.implicitly_wait(2)
        count += 1

        try:
            # fetching all the rows in page
            college_elements = driver.find_elements(By.TAG_NAME,'tr')
    
            for college_ele in college_elements:
                # then for every element in the row extracting the table data
                college_data = college_ele.find_elements(By.TAG_NAME,'td')
            
                # removing tha new line and replacing with space
                row_data=[d.text.strip().replace("\n"," ") for d in college_data]
                row_data=[row for row in row_data if row]
                # checking if data is there and the sr containg '#'
                if row_data:
                    if "#" in row_data[0]:
                        csv_writer.writerow(row_data)
            
            # creating a break condition
            if target_scroll in driver.page_source:
                print(count)
                break
        
        except Exception as e:
            print(e)

end = time.time()
execution_time = end - start
print(f"Execution time: {execution_time} seconds")




In [ ]:
driver.quit()

<h3> Uploading data to hdfs using pyspark

In [ ]:
from pyspark.sql import SparkSession
import os

setting ip address

In [ ]:
os.environ["SPARK_IP"]="10.0.2.15"

In [ ]:
spark=SparkSession.builder.appName("webscraping").getOrCreate()

In [ ]:
try:
    df=spark.read.csv("/home/hdoop/Downloads/college_dunia.csv",inferSchema=True,header=True)
    new_df=df.dropDuplicates()
    new_df.write.format("csv").save('hdfs://localhost:9000/web_scraper/scrapped_data.csv',header=True,inferschema=True)
except Exception as e:
    print(e)